<a href="https://colab.research.google.com/github/sriku2412/GneAI---Digital-Earth-Gardians/blob/main/clustering_catigorising_ranking__using_genAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = pd.read_csv(r'https://raw.githubusercontent.com/sriku2412/GneAI---Digital-Earth-Gardians/main/AI%20EarthHack%20Dataset.csv', encoding = "ISO-8859-1")
data

,id,problem,solution
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m..."
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single..."
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa..."
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...
...,...,...,...
1295,1296,"The linear 'take, make, dispose' model of prod...","Addressing this problem, a two-pronged circula..."
1296,1297,The conundrum we face is the improper disposal...,The proposed solution is a cutting-edge recycl...
1297,1298,This solution will help the vegetation,Use old tea bags as compost for soil
1298,1299,Accumulation and improper disposal of single-u...,My proposed solution is to implement an innova...


In [2]:
!pip install sentence-transformers
!pip install openai==0.28
!pip install --upgrade typing_extensions
!pip install langchain_community

In [3]:
import os
# Set up OpenAI API key securely
api_key = 'OPENAI_KEY'  # Replace with your actual API key
os.environ["OPENAI_API_KEY"] = api_key

In [4]:
import openai



# Function to determine if a text is related to circular economy
def is_related_to_circular_economy(text):
    response = openai.Completion.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f"Is the following text related to the circular economy? Yes or No.\n\nText: {text}",
      temperature=0,
      max_tokens=60
    )
    return response.choices[0].text.strip()


# Function to handle non-string values
def concatenate_problem_solution(row):
    problem = str(row['problem']) if not pd.isna(row['problem']) else ''
    solution = str(row['solution']) if not pd.isna(row['solution']) else ''
    return problem + ' ' + solution

# Apply the function to each row
data['circular_economy'] = data.apply(lambda row: is_related_to_circular_economy(concatenate_problem_solution(row)), axis=1)

data.head()


,id,problem,solution,circular_economy
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",Yes
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",Yes
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",Yes
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,Yes
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,Yes


In [5]:
data['circular_economy'].value_counts()

Yes                                                                                                                                                                                                                                                                                                                  1215
No                                                                                                                                                                                                                                                                                                                     75
.\n\nYes                                                                                                                                                                                                                                                                                                                5
,\n\nNo                                                   

In [6]:
data = data[data['circular_economy'] != 'no']

In [7]:
data.head()

,id,problem,solution,circular_economy
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",Yes
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",Yes
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",Yes
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,Yes
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,Yes


In [8]:
data = data[['problem','solution']]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Extract the text data for clustering
text_data = data['problem']

# Create a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Transform the text data into TF-IDF features
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)

# Perform KMeans clustering
n_clusters = 6
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Assign the cluster labels to the original dataframe
data['cluster'] = kmeans.labels_

# Display the first few rows of the dataframe with cluster labels
data.head()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,problem,solution,cluster
0,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",5
1,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",5
2,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",5
3,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,1
4,The majority of the materials used in producin...,An innovative concept would be a modular elect...,4


In [10]:
import openai
import pandas as pd

def classify_problems_with_lambda(data, model="gpt-3.5-turbo-instruct"):
    categories = {
        0: "food wastage",
        1: "fashion industry pollution",
        2: "Construction and digital platform impact",
        3: "Electronic waste and resources utilization",
        4: "plastic waste and usage",
        5: "others"
    }

    def get_category(problem):
        prompt = f"Classify the following problem into one of the categories: {categories}\n\nProblem: {problem}\nCategory:"
        response = openai.Completion.create(
            model=model,
            prompt=prompt,
            max_tokens=60,
            temperature=0.3
        )
        response_text = response.choices[0].text.strip()

        for cat_num, cat_desc in categories.items():
            if cat_desc in response_text:
                return cat_num
        return 5  # Default to "others" if no match is found

    # Use a lambda function to apply 'get_category' to each problem
    data['category'] = data['problem'].apply(lambda problem: get_category(problem))
    return data

In [11]:
data = classify_problems_with_lambda(data)

In [12]:
categories = {
        0: "food wastage",
        1: "fashion industry pollution",
        2: "Construction and digital platform impact",
        3: "Electronic waste and resources utilization",
        4: "plastic waste and usage",
        5: "others"
    }

data['category'] = data['category'].map(categories)
data.head()

,problem,solution,cluster,category
0,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",5,Construction and digital platform impact
1,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",5,Electronic waste and resources utilization
2,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",5,Construction and digital platform impact
3,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,1,fashion industry pollution
4,The majority of the materials used in producin...,An innovative concept would be a modular elect...,4,Electronic waste and resources utilization


In [13]:
import pandas as pd
import openai
import re  # Import regular expressions for extracting numbers


def extract_first_number(text):
    """
    Extracts the first number found in the given text.
    """
    match = re.search(r'\d+', text)
    return match.group(0) if match else None

def rate_solutions(solution):
    # Define prompts for each aspect
    prompts = {
        "Radical Innovation": "Rate the following solution for its radical innovation on a scale from 1 to 10:",
        "Potential for Transformative Impact": "Rate the following solution for its potential for transformative impact on a scale from 1 to 10:",
        "Scalability and Global Reach": "Rate the following solution for its scalability and global reach on a scale from 1 to 10:",
        "Feasibility in the Foreseeable Future": "Rate the following solution for its feasibility in the foreseeable future on a scale from 1 to 10:"
    }

    ratings = {}

    for aspect, prompt in prompts.items():
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",
            prompt=f"{prompt}\n\n{solution}\n\nRating:",
            max_tokens=10
        )
        # Extract only the first number from the response
        rating = extract_first_number(response.choices[0].text.strip())
        ratings[aspect] = rating

    return ratings

# Apply the function to the 'solution' column and split the results into four new columns
data[['Radical Innovation', 'Potential for Transformative Impact', 'Scalability and Global Reach', 'Feasibility in the Foreseeable Future']] = data['solution'].apply(
    lambda x: pd.Series(rate_solutions(x))
)


In [14]:
# Convert the four columns from string to numeric
data['Feasibility in the Foreseeable Future'] = pd.to_numeric(data['Feasibility in the Foreseeable Future'], errors='coerce')
data['Potential for Transformative Impact'] = pd.to_numeric(data['Potential for Transformative Impact'], errors='coerce')
data['Radical Innovation'] = pd.to_numeric(data['Radical Innovation'], errors='coerce')
data['Scalability and Global Reach'] = pd.to_numeric(data['Scalability and Global Reach'], errors='coerce')

# Calculate the 'score' as the sum of the four columns
data['score'] = (data['Feasibility in the Foreseeable Future'] +
                 data['Potential for Transformative Impact'] +
                 data['Radical Innovation'] +
                 data['Scalability and Global Reach'])
data


,problem,solution,cluster,category,Radical Innovation,Potential for Transformative Impact,Scalability and Global Reach,Feasibility in the Foreseeable Future,score
0,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",5,Construction and digital platform impact,9.0,9.0,9.0,9.0,36.0
1,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",5,Electronic waste and resources utilization,9.0,9.0,8.0,8.0,34.0
2,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",5,Construction and digital platform impact,9.0,8.0,9.0,9.0,35.0
3,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,1,fashion industry pollution,9.0,9.0,8.0,9.0,35.0
4,The majority of the materials used in producin...,An innovative concept would be a modular elect...,4,Electronic waste and resources utilization,9.0,10.0,9.0,9.0,37.0
...,...,...,...,...,...,...,...,...,...
1295,"The linear 'take, make, dispose' model of prod...","Addressing this problem, a two-pronged circula...",5,Construction and digital platform impact,9.0,9.0,8.0,8.0,34.0
1296,The conundrum we face is the improper disposal...,The proposed solution is a cutting-edge recycl...,5,others,9.0,9.0,8.0,8.0,34.0
1297,This solution will help the vegetation,Use old tea bags as compost for soil,5,others,7.0,9.0,5.0,7.0,28.0
1298,Accumulation and improper disposal of single-u...,My proposed solution is to implement an innova...,2,plastic waste and usage,9.0,9.0,9.0,9.0,36.0


In [16]:

# Grouping by category and creating a new column 'rank' that ranks the score in order within each category
data['rank'] = data.groupby('category')['score'].rank("dense", ascending=False)

# Replace non-finite values (NaN or inf) with a specific value (e.g., 0)
data['rank'] = data['rank'].fillna(0).astype(int)

# Now, your 'rank' column should contain integer values without non-finite values.


In [17]:
# Specify the file path where you want to save the CSV file, including the file name
file_path = '/content/GneAI---Digital-Earth-Gardians/AI_EarthHack_Dataset_2.csv'

# Write the DataFrame to a CSV file
data.to_csv(file_path, index=False)  # Set index=False to exclude row numbers

# Confirm that the file has been created
!ls '/content/GneAI---Digital-Earth-Gardians/'  # List files in the specified directory

 AI_EarthHack_Dataset_2.csv   GneAI---Digital-Earth-Gardians
'AI EarthHack Dataset.csv'    Untitled17.ipynb
